In [ ]:
import threading
import cv2
import face_recognition
from concurrent.futures import ThreadPoolExecutor

# Load the reference image
reference_image = face_recognition.load_image_file("WIN_20231224_17_43_55_Pro.jpg")
reference_face_encoding = face_recognition.face_encodings(reference_image)[0]

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

face_match = False
confidence_threshold = 0.6  # Adjust this threshold based on your observations

# Initialize ThreadPoolExecutor with a single worker
executor = ThreadPoolExecutor(max_workers=1)
lock = threading.Lock()


def detect_face(frame):
    # Resize the frame to match the reference image dimensions
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    face_locations = face_recognition.face_locations(small_frame, model="hog")
    return face_locations


def check_face(frame):
    global face_match
    try:
        # Resize the frame to match the reference image dimensions
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        face_locations = face_recognition.face_locations(small_frame, model="hog")

        if not face_locations:
            face_match = False
            return

        face_encoding = face_recognition.face_encodings(small_frame, face_locations)[0]

        # Compare face embeddings
        results = face_recognition.compare_faces([reference_face_encoding], face_encoding, tolerance=confidence_threshold)
        face_match = results[0]
    except Exception as e:
        print(f"Error in face verification: {e}")
        face_match = False


def process_frame(frame):
    global face_match

    # Face detection
    face_locations = detect_face(frame)

    # Submit the face recognition task to the ThreadPoolExecutor
    future = executor.submit(check_face, frame)

    # Draw a square around the detected face(s)
    for (top, right, bottom, left) in face_locations:
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 5)

    if face_match:
        cv2.putText(frame, "Face Match", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "No Face Match", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("frame", frame)


while True:
    ret, frame = cap.read()

    if ret:
        process_frame(frame)

    if cv2.waitKey(1) == ord("q"):
        # Shutdown the ThreadPoolExecutor when exiting the loop
        executor.shutdown()
        break

cv2.destroyAllWindows()
